In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

import tensorflow as tf
import numpy as np

import time

import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from notebooks.helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstraping_partitions_full_int_q,
    )

In [12]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.
Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [13]:
x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_dataset, data_type="time-series")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_dataset, data_type="time-series")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_dataset, data_type="time-series")

In [35]:
BASE_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 47996, 4)          24        
                                                                 
 max_pooling1d_21 (MaxPooli  (None, 5999, 4)           0         
 ng1D)                                                           
                                                                 
 conv1d_22 (Conv1D)          (None, 5995, 4)           84        
                                                                 
 max_pooling1d_22 (MaxPooli  (None, 749, 4)            0         
 ng1D)                                                           
                                                                 
 conv1d_23 (Conv1D)          (None, 745, 8)            168       
                                                                 
 max_pooling1d_23 (MaxPooli  (None, 93, 8)           

In [36]:
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_test_np, y_test_np, model_format="keras")


44/44 [==============================] - 2s 44ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9770420356059069

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9774609849814141


### Convert to TF Lite format

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmphdaeg9p5/assets


INFO:tensorflow:Assets written to: /tmp/tmphdaeg9p5/assets
2024-09-22 18:07:15.948157: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 18:07:15.948407: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 18:07:15.949104: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmphdaeg9p5
2024-09-22 18:07:15.952732: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 18:07:15.952766: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmphdaeg9p5
2024-09-22 18:07:15.956963: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 18:07:16.013770: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmphdaeg9p5
2024-09-22 18:07:16.035012: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [21]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/cnn/quantized/cnn_time_series.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.26%
Recall: 96.61%
Precision: 98.28%
F1-score: 97.44%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9748849299738644

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9733967700475985

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9770420356059069

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9774812327623107

Time for Test dataset:
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.012 seconds


In [22]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 44.035 Kilobytes


'44.035 Kilobytes'

### Dynamic range quantization

In [23]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp7y5rlzl0/assets


INFO:tensorflow:Assets written to: /tmp/tmp7y5rlzl0/assets
2024-09-22 18:07:45.142103: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 18:07:45.142159: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 18:07:45.142362: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp7y5rlzl0
2024-09-22 18:07:45.143758: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 18:07:45.143773: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp7y5rlzl0
2024-09-22 18:07:45.147043: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 18:07:45.201781: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp7y5rlzl0
2024-09-22 18:07:45.220004: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [24]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/cnn/quantized/cnn_time_series_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.26%
Recall: 96.61%
Precision: 98.28%
F1-score: 97.44%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9748849299738644

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9706781288304808

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9770420356059069

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9778918475491608

Time for Test dataset:
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.01 seconds
M

In [25]:
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

File size: 17.953 Kilobytes


'17.953 Kilobytes'

### Float16 quantization

In [26]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp3b_m9n4e/assets


INFO:tensorflow:Assets written to: /tmp/tmp3b_m9n4e/assets
2024-09-22 18:08:18.104493: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 18:08:18.104575: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 18:08:18.104761: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp3b_m9n4e
2024-09-22 18:08:18.106884: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 18:08:18.106914: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp3b_m9n4e
2024-09-22 18:08:18.110474: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 18:08:18.163186: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp3b_m9n4e
2024-09-22 18:08:18.197997: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [27]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "../../models/time_series_models_from_notebooks/cnn/quantized/cnn_time_series_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.26%
Recall: 96.61%
Precision: 98.28%
F1-score: 97.44%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9748849299738644

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.97684602966927

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9770420356059069

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9802228826492851

Time for Test dataset:
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.014 seconds
Mi

In [28]:
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

File size: 27.406 Kilobytes


'27.406 Kilobytes'

### Full integer quantization

In [29]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(5000):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

full_int_model = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp5dixik_g/assets


INFO:tensorflow:Assets written to: /tmp/tmp5dixik_g/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-09-22 18:08:54.574566: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-22 18:08:54.574611: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-22 18:08:54.574767: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp5dixik_g
2024-09-22 18:08:54.576262: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-22 18:08:54.576275: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp5dixik_g
2024-09-22 18:08:54.580656: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-22 18:08

In [30]:
interpreter = tf.lite.Interpreter(model_content=full_int_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [31]:
import pathlib

tflite_models_dir = pathlib.Path("../../models/time_series_models_from_notebooks/cnn/quantized/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_full_int_q_file = tflite_models_dir/"cnn_time_series_full_int_q.tflite"
tflite_model_full_int_q_file.write_bytes(full_int_model)

19360

In [32]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

# print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
# non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
# print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

# print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
# bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_q_file, x_val_np, y_val_np)
# print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_full_int_q_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

# print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
# non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
# print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

# print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
# bootstrap_patritions_f1_scores = get_f1_scores_of_bootstraping_partitions_full_int_q(tflite_model_full_int_q_file, x_test_np, y_test_np)
# print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_full_int_q_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_full_int_q_file), "KB")

Validation dataset:
Accuracy: 91.01%
Recall: 73.94%
Precision: 99.71%
F1-score: 84.91%

Test dataset:
Accuracy: 88.30%
Recall: 64.63%
Precision: 99.66%
F1-score: 78.41%

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.009 seconds
Min: 0.002 seconds
File size: 18.906 Kilobytes


'18.906 Kilobytes'